In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import AutoProcessor, Wav2Vec2Model
import torch
from datasets import load_dataset

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

audio_processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
audio_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

# audio file is decoded on the fly
inputs = audio_processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    outputs = audio_model(**inputs)

last_hidden_states = outputs.last_hidden_state.squeeze(0)
embeddings = last_hidden_states.mean(dim=0)
list(embeddings.shape)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[768]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text_model = AutoModel.from_pretrained("bert-base-uncased")

sentence = "This is a test sentence."
inputs = tokenizer(sentence, return_tensors="pt")
outputs = text_model(**inputs)

embeddings = outputs.last_hidden_state.squeeze(0)
pooled_embeddings = embeddings.mean(dim=0)
pooled_embeddings.size()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([768])

In [ ]:
import torch
import torch.nn as nn

class JointNN(nn.Module):
    def __init__(self, in_features):
        super(MyNet, self).__init__()
        self.text_seq = nn.Sequential(
            nn.Linear(in_features, 576),
            nn.BatchNorm1d(576),
            nn.LeakyReLU(),
            nn.Linear(576, 384),
        )
        self.audio_seq = nn.Sequential(
            nn.Linear(in_features, 576),
            nn.BatchNorm1d(576),
            nn.LeakyReLU(),
            nn.Linear(576, 384),
        )

    def forward(self, x_text, x_audio):
        x1 = self.text_seq(x_text)
        x2 = self.audio_seq(x_audio)
        return x1, x2


In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-lirc3lp1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-lirc3lp1
  Resolved https://github.com/huggingface/transformers to commit 3dae0d7b4fb8d7e9383b893e4e1799191bb2ab7b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset
from transformers import AutoProcessor, ClapModel

dataset = load_dataset("ashraq/esc50")
audio_sample = dataset["train"]["audio"][0]["array"]

model = ClapModel.from_pretrained("laion/clap-htsat-unfused")
processor = AutoProcessor.from_pretrained("laion/clap-htsat-unfused")

input_text = ["Sound of a dog", "Sound of vaccum cleaner"]

inputs = processor(text=input_text, audios=audio_sample, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_audio = outputs.logits_per_audio  # this is the audio-text similarity score
probs = logits_per_audio.softmax(dim=-1)  # we can take the softmax to get the label probabilities

  0%|          | 0/1 [00:00<?, ?it/s]

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
dataset = load_dataset("asapp/slue", "voxpopuli")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
dataset['train']

Dataset({
    features: ['id', 'audio', 'speaker_id', 'normalized_text', 'raw_text', 'raw_ner', 'normalized_ner', 'raw_combined_ner', 'normalized_combined_ner'],
    num_rows: 5000
})

In [ ]:
texts = dataset['train']['normalized_text'][:50]
audios = [x["array"] for x in dataset['train']['audio'][:50]]

inputs = processor(text=texts, audios=audios, return_tensors="pt", padding=True)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
from transformers import TextDataset
from transformers import Wav2Vec2Dataset


train_dataset = Wav2Vec2Dataset(inputs=inputs)

text_dataset = TextDataset(
    # A list of text strings
    text_list=texts,
    tokenizer=processor.tokenizer
)

# Combine the image and text datasets into a single dataset
finetune_dataset = [(image_features, text_input) for image_features, text_input in zip(train_dataset, text_dataset)]


In [ ]:
from torch.utils.data import TensorDataset
import torch

finetune_dataset = TensorDataset(
    torch.tensor(inputs['input_ids']),  # Pass the `input_ids` key as input data
    torch.tensor(inputs['attention_mask']),  # Pass the `attention_mask` key as attention mask data
    torch.tensor(inputs['input_features'])
)

<ipython-input-13-08d545182e4a>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(inputs['input_ids']),  # Pass the `input_ids` key as input data
<ipython-input-13-08d545182e4a>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(inputs['attention_mask']),  # Pass the `attention_mask` key as attention mask data
<ipython-input-13-08d545182e4a>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(inputs['input_features'])


In [ ]:

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,            # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


trainer = Trainer(
    model=model,                      # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=finetune_dataset,         # training dataset
)

trainer.train()

In [ ]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding